# Segmenting and Clustering Neighborhoods in Toronto (Part 1)

### 1. Importing Libraries

In [105]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize 
import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

print("Done!")

Done!


### 2. Scrap the Data from the Wikipedia page, and transform the data into a pandas dataframe

Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [58]:
Data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(Data, 'html.parser')

In [59]:
# create three empty lists to store table data
postalCode = []
borough = []
neighborhood = []

In [60]:
# find the table
soup.find('table').find_all('tr')

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCode.append(cells[0].text)
        borough.append(cells[1].text)
        neighborhood.append(cells[2].text) # avoid new lines in neighborhood cell

In [63]:
# create a new DataFrame from the three lists
toronto = pd.DataFrame({"PostalCode": postalCode,
                           "Borough": borough,
                           "Neighborhood": neighborhood})

toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


### 3. Ignore cells with a borough that is Not assigned

In [64]:
toronto_dropna = toronto[toronto.Borough != "Not assigned\n"].reset_index(drop=True)
toronto_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


### 4. Group neighborhoods in the same borough
Note: If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [65]:
toronto_grouped = toronto_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill\n"
3,M1G\n,Scarborough\n,Woburn\n
4,M1H\n,Scarborough\n,Cedarbrae\n
5,M1J\n,Scarborough\n,Scarborough Village\n
6,M1K\n,Scarborough\n,"Kennedy Park, Ionview, East Birchmount Park\n"
7,M1L\n,Scarborough\n,"Golden Mile, Clairlea, Oakridge\n"
8,M1M\n,Scarborough\n,"Cliffside, Cliffcrest, Scarborough Village West\n"
9,M1N\n,Scarborough\n,"Birch Cliff, Cliffside West\n"


In [66]:
for index, row in toronto_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned\n":
        row["Neighborhood"] = row["Borough"]
        
toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge\n"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek\n"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill\n"
3,M1G\n,Scarborough\n,Woburn\n
4,M1H\n,Scarborough\n,Cedarbrae\n


In [91]:
toronto_grouped['PostalCode'] = toronto_grouped['PostalCode'].str.replace(r'\n', '')
toronto_grouped['Borough'] = toronto_grouped['Borough'].str.replace(r'\n', '')
toronto_grouped['Neighborhood'] = toronto_grouped['Neighborhood'].str.replace(r'\n', '')
toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### 5. Print the number of rows of the dataframe

In [92]:
toronto_grouped.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto (Part 2)

### 1. Load the coordinates from the csv file

In [93]:
coord = pd.read_csv("Geospatial_Coordinates.csv")
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 2. cleaning the coordinate table

In [94]:
# rename the column "Postal Code" into "PostalCode"
coord.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
coord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### 3. Merge two tables

In [96]:
# merge tables on the column "PostalCode"
toronto_merged = toronto_grouped.merge(coord, on="PostalCode", how="left")
toronto_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Segmenting and Clustering Neighborhoods in Toronto (Part 3)

### 1. Geting Toronto latitude and longitude Values

In [100]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [101]:
from geopy.geocoders import Nominatim

In [103]:
city = 'Toronto'

geolocator = Nominatim(user_agent="my-map")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


### 2. Drop everything except Toronto neighborhoods

In [109]:
toronto_data = toronto_merged[toronto_merged.Borough.str.contains("Toronto")].reset_index(drop = True)
toronto_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


### 3. Check neighborhood data

In [110]:
toronto_data.loc[0, 'Neighborhood']

neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


### 4. Contruct API calls and request JSON

In [142]:
CLIENT_ID = '*****' # your Foursquare ID
CLIENT_SECRET = '*****' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: *****
CLIENT_SECRET:*****


In [143]:
# Construct URL
radius = 500
LIMIT = 100
# Constructed API calls
url = 'https://api.foursquare.com/v2/venues/explore?client_id=****&client_secret=*****&ll=43.6534817,-79.3839347&v=20180605&radius=500&limit=100'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
# Request JSON
results = requests.get(url).json()

In [114]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### 5. Clean JSON to get categories

In [115]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Indigo,Bookstore,43.653515,-79.380696
3,Poke Guys,Poke Place,43.654895,-79.385052
4,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684


### 6. Use function to get nearby venues of the neighborhood

In [118]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id=******&client_secret=*******&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'.format(
             CLIENT_ID, 
             CLIENT_SECRET, 
             VERSION, 
             neighborhood_latitude, 
             neighborhood_longitude, 
             radius, 
             LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [119]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

90 venues were returned by Foursquare.


### 7. Get all nearby venues from each neighborhoods

In [120]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [121]:
print(toronto_venues.shape)
toronto_venues.head()

(3510, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,The Beaches,43.676357,-79.293031,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,The Beaches,43.676357,-79.293031,Indigo,43.653515,-79.380696,Bookstore
3,The Beaches,43.676357,-79.293031,Poke Guys,43.654895,-79.385052,Poke Place
4,The Beaches,43.676357,-79.293031,Chatime 日出茶太,43.655542,-79.384684,Bubble Tea Shop


In [122]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 63 uniques categories.


### 8. Analyze Each Area

In [123]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'], inplace=True, axis = 1)
#toronto_onehot['Neighborhood']
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.columns
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + toronto_onehot.columns[:-1].tolist()
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,General Travel,Gym,Gym / Fitness Center,Hotel,Japanese Restaurant,Jazz Club,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Women's Store
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [124]:
toronto_onehot.shape

(3510, 63)

### 9. Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [125]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food Court,Furniture / Home Store,Gastropub,General Travel,Gym,Gym / Fitness Center,Hotel,Japanese Restaurant,Jazz Club,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Pizza Place,Plaza,Poke Place,Ramen Restaurant,Restaurant,Salad Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Women's Store
0,Berczy Park,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.066667,0.011111,0.077778,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.033333,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
1,"Brockton, Parkdale Village, Exhibition Place",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.066667,0.011111,0.077778,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.033333,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
2,"Business reply mail Processing Centre, South C...",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.066667,0.011111,0.077778,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.033333,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.066667,0.011111,0.077778,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.033333,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
4,Central Bay Street,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.066667,0.011111,0.077778,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.033333,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.022222,0.011111,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
5,Christie,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.066667,0.0111

In [126]:
toronto_grouped.shape

(39, 63)

### 10. Print each neighborhood along with the top 5 most common venues

In [127]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Brockton, Parkdale Village, Exhibition Place----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2        

### 11. Let's put that into a pandas dataframe

In [128]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [129]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office


### 12. Cluster Neighborhoods using k-means

In [135]:
from sklearn.cluster import KMeans

In [131]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:37]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  import sys


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [132]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.astype({'Cluster Labels': 'int64'})

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office


In [136]:
!pip install folium

In [140]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters